In [51]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Ejercicio 1
Parte 1 (imágenes en /white_patchy /coord_cromaticas)
1) Implementar el algoritmo de pasaje a coordenadas cromáticas para librarnos de las variaciones de contraste

In [53]:
# Cargar las imágenes 
imagen_original_1 = cv2.imread('coord_cromaticas/CoordCrom_1.png')

imagen_original_2 = cv2.imread('coord_cromaticas/CoordCrom_2.png')

imagen_original_3 = cv2.imread('coord_cromaticas/CoordCrom_3.png')


In [54]:
# función para pasaje a coordenadas cromáticas
def coord_cromatica (imagen):
   
    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
  
    
    # Separa los canales R, G, B
    R = imagen_rgb[:, :, 0].astype(float)
    G = imagen_rgb[:, :, 1].astype(float)
    B = imagen_rgb[:, :, 2].astype(float)
    
    # Calcula la suma de los canales
    suma = R + G + B   
    # Inicializa r, g, b con ceros
    r = np.zeros_like(R)
    g = np.zeros_like(G)
    b = np.zeros_like(B)  

    # Calcula las coordenadas cromaticas donde la suma no es cero
    # Usa una mascara para evitar la division por cero
    mascara = suma > 0
    r[mascara] = R[mascara] / suma[mascara]
    g[mascara] = G[mascara] / suma[mascara]
    b[mascara] = B[mascara] / suma[mascara]

    # Crea la imagen en coordenadas cromaticas
    imagen_cromatica = np.stack([r, g, b], axis=-1)

    return imagen_rgb, imagen_cromatica 
    

In [55]:
def mostrar_coor_crom(imagen_rgb, imagen_cromatica):
  
    # Muestra la imagen original y la imagen en coordenadas cromaticas
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.title("Imagen Original")
    plt.imshow(imagen_rgb)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Imagen en Coordenadas Cromáticas")
    plt.imshow(imagen_cromatica)
    plt.axis('off')

    plt.show()

In [ ]:

imagen_rgb, imagen_cromatica = coord_cromatica(imagen_original_1)
mostrar_coor_crom(imagen_rgb, imagen_cromatica)


In [ ]:
imagen_rgb, imagen_cromatica = coord_cromatica(imagen_original_2)
mostrar_coor_crom(imagen_rgb, imagen_cromatica)

In [ ]:
imagen_rgb, imagen_cromatica = coord_cromatica(imagen_original_3)
mostrar_coor_crom(imagen_rgb, imagen_cromatica)

2. Implementar el algoritmo White Patch para librarnos de las diferencias de color de iluminación.

In [59]:
import glob

# Carga de la carpet que contiene las imágenes png
folder_path = './white_patch/*.png'  

# Lista para almacenar las imágenes
images = []

# Usar glob para encontrar las imágenes con el path relativo
for img_path in glob.glob(folder_path):
    # Cargar la imagen
    img = cv2.imread(img_path)
    # Verificar si la imagen se cargó correctamente
    if img is not None:
        images.append(img)

In [60]:
def white_patch(imagen):
   
    # Convierte la imagen de BGR (por defecto en OpenCV) a RGB
    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

    # Separa los canales R, G, B
    R = imagen_rgb[:, :, 0].astype(float)
    G = imagen_rgb[:, :, 1].astype(float)
    B = imagen_rgb[:, :, 2].astype(float)

    # Encuentra los valores maximos de cada canal
    max_R = np.max(R)
    max_G = np.max(G)
    max_B = np.max(B)

    # Calcula los factores de correccion
    escala_R = 255.0 / max_R
    escala_G = 255.0 / max_G
    escala_B = 255.0 / max_B 

    # Aplica los factores de correccion
    R = np.clip(R * escala_R, 0, 255).astype(np.uint8)
    G = np.clip(G * escala_G, 0, 255).astype(np.uint8)
    B = np.clip(B * escala_B, 0, 255).astype(np.uint8)

    # Recombina los canales
    imagen_balanceada = cv2.merge([R, G, B])

    return imagen_rgb, imagen_balanceada

In [61]:
def mostrar_white_patch(imagen_rgb, imagen_balanceada):
   
    # Muestra la imagen original y la imagen con white patch
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.title("Imagen Original")
    plt.imshow(imagen_rgb)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Imagen con White Patch")
    plt.imshow(imagen_balanceada)
    plt.axis('off')

    plt.show()

In [ ]:
# Mostrar las imágenes
for idx, img in enumerate(images):
    rgb,balanceada = white_patch(img)
    mostrar_white_patch(rgb,balanceada)
   
    

In [49]:
import glob

# Carga de la carpet que contiene las imágenes jpg
folder_path = './white_patch/*.jpg'  

# Lista para almacenar las imágenes
images = []

# Usar glob para encontrar las imágenes con el path relativo
for img_path in glob.glob(folder_path):
    # Cargar la imagen
    img = cv2.imread(img_path)
    # Verificar si la imagen se cargó correctamente
    if img is not None:
        images.append(img)

In [ ]:
# Mostrar las imágenes
for idx, img in enumerate(images):
    rgb,balanceada = white_patch(img)
    mostrar_white_patch(rgb,balanceada)

3. Mostrar los resultados obtenidos y analizar las posibles fallas (si es que las hay) en el caso de White patch.


Imagen de las cuatro manos: 

    El algoritmo logra su objetivo, ajustando los colores para que los objetos blancos se vean realmente blancos. Esto se basa en encontrar un píxel cercano al blanco puro (máximos valores RGB) y usarlo para ajustar el resto de la imagen. En este caso, la pared blanca de fondo permite que el algoritmo funcione correctamente.

Imagen del Depredador: 

    El algoritmo funciona mejor en la imagen con luz roja, probablemente porque identifica más fácilmente un valor máximo en el canal rojo y ajusta el balance de blancos en consecuencia. Además, el ojo humano percibe mejor el rojo y verde que el azul, por lo que la corrección bajo luz roja parece más natural. La diferencia en los resultados entre formatos PNG y JPG se debe a que el JPG usa compresión con pérdida, lo que afecta los colores, mientras que el PNG mantiene los datos originales, permitiendo una corrección más precisa.